In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem, Descriptors, MolFromSmiles
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [2]:
# Import main data and get list of SMILES
molecules = pd.read_csv('./fathead_minnow_dataset.csv')  # Load the photoswitch dataset using pandas
smiles_list = list(molecules.SMILES.values)

In [3]:
len(smiles_list)

554

In [7]:
# Initiate list of rdkit molecules
rdkit_mols = [MolFromSmiles(smiles) for smiles in smiles_list]

In [8]:
# Get Morgan fingerprints, note the parameters!
morgan_fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, radius = 3, nBits = 2048) for mol in rdkit_mols]
morgan_fingerprints = np.asarray(morgan_fingerprints)

In [17]:
# Turn into pandas dataframe and add smiles as a first column
morgan_fingerprints = pd.DataFrame(data = morgan_fingerprints)
morgan_fingerprints.insert(0, 'SMILES', smiles_list)

In [18]:
morgan_fingerprints

,SMILES,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,N#CCC#N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,C1COCCO1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CN(C)N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,C1=CC=CC(O)=C1C(=O)OC2=CC=CC=C2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ClCCCCl,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,C1=CC=CC=C1CCCCC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
550,CN(C)C1=CC=C(C=O)C=C1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
551,C1=C(N)C=CC=C1OCC2=CC=CC=C2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
552,CC(C)SSC(C)C,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
morgan_fingerprints.to_csv('morgan_fingerprints.csv')

In [20]:
# Next, rdkit's own descriptors
from rdkit.Chem import Descriptors

In [21]:
# A list of desriptors
Descriptors.descList

[('MaxEStateIndex',
  <function rdkit.Chem.EState.EState.MaxEStateIndex(mol, force=1)>),
 ('MinEStateIndex',
  <function rdkit.Chem.EState.EState.MinEStateIndex(mol, force=1)>),
 ('MaxAbsEStateIndex',
  <function rdkit.Chem.EState.EState.MaxAbsEStateIndex(mol, force=1)>),
 ('MinAbsEStateIndex',
  <function rdkit.Chem.EState.EState.MinAbsEStateIndex(mol, force=1)>),
 ('qed',
  <function rdkit.Chem.QED.qed(mol, w=QEDproperties(MW=0.66, ALOGP=0.46, HBA=0.05, HBD=0.61, PSA=0.06, ROTB=0.65, AROM=0.48, ALERTS=0.95), qedProperties=None)>),
 ('MolWt', <function rdkit.Chem.Descriptors.<lambda>(*x, **y)>),
 ('HeavyAtomMolWt', <function rdkit.Chem.Descriptors.HeavyAtomMolWt(x)>),
 ('ExactMolWt', <function rdkit.Chem.Descriptors.<lambda>(*x, **y)>),
 ('NumValenceElectrons',
  <function rdkit.Chem.Descriptors.NumValenceElectrons(mol)>),
 ('NumRadicalElectrons',
  <function rdkit.Chem.Descriptors.NumRadicalElectrons(mol)>),
 ('MaxPartialCharge',
  <function rdkit.Chem.Descriptors.MaxPartialCharge(mo

In [22]:
# Write a dictionary of name:function pairs for all descriptors
all_descriptors = {d[0]: d[1] for d in Descriptors.descList}

In [23]:
# Initialise a new pandas df
rdkit_descriptors = pd.DataFrame(data = {'SMILES': np.array((smiles_list)) })
rdkit_descriptors

,SMILES
0,N#CCC#N
1,C1COCCO1
2,CN(C)N
3,C1=CC=CC(O)=C1C(=O)OC2=CC=CC=C2
4,ClCCCCl
...,...
549,C1=CC=CC=C1CCCCC
550,CN(C)C1=CC=C(C=O)C=C1
551,C1=C(N)C=CC=C1OCC2=CC=CC=C2
552,CC(C)SSC(C)C


In [24]:
# Compute each descriptor (outer loop) for each molecule(inside)
for feature in all_descriptors:
    values = []
    for mol in rdkit_mols:
        values += [all_descriptors[feature](mol)]
    rdkit_descriptors[feature] = values

rdkit_descriptors

/var/folders/kg/nxk5934n7y5dzv72d7zk_4th0000gn/T/ipykernel_5783/2319821685.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rdkit_descriptors[feature] = values
/var/folders/kg/nxk5934n7y5dzv72d7zk_4th0000gn/T/ipykernel_5783/2319821685.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rdkit_descriptors[feature] = values
/var/folders/kg/nxk5934n7y5dzv72d7zk_4th0000gn/T/ipykernel_5783/2319821685.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

/var/folders/kg/nxk5934n7y5dzv72d7zk_4th0000gn/T/ipykernel_5783/2319821685.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rdkit_descriptors[feature] = values
/var/folders/kg/nxk5934n7y5dzv72d7zk_4th0000gn/T/ipykernel_5783/2319821685.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rdkit_descriptors[feature] = values
/var/folders/kg/nxk5934n7y5dzv72d7zk_4th0000gn/T/ipykernel_5783/2319821685.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,N#CCC#N,7.593750,0.000000,7.593750,0.000000,0.411164,66.063,64.047,66.021798,24,...,0,0,0,0,0,0,0,0,0,0
1,C1COCCO1,4.944444,0.777778,4.944444,0.777778,0.415749,88.106,80.042,88.052429,36,...,0,0,0,0,0,0,0,0,0,0
2,CN(C)N,4.944444,1.500000,4.944444,1.500000,0.299219,60.100,52.036,60.068748,26,...,0,0,0,0,0,0,0,0,0,0
3,C1=CC=CC(O)=C1C(=O)OC2=CC=CC=C2,11.652689,-0.565463,11.652689,0.080156,0.617037,214.220,204.140,214.062994,80,...,0,0,0,0,0,0,0,0,0,0
4,ClCCCCl,5.217207,0.684028,5.217207,0.684028,0.479095,112.987,106.939,111.984656,32,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,C1=CC=CC=C1CCCCC,2.241065,1.244722,2.241065,1.244722,0.573822,148.249,132.121,148.125201,60,...,0,0,0,0,0,0,0,0,1,0
550,CN(C)C1=CC=C(C=O)C=C1,10.274032,0.718148,10.274032,0.718148,0.594673,149.193,138.105,149.084064,58,...,0,0,0,0,0,0,0,0,0,0
551,C1=C(N)C=CC=C1OCC2=CC=CC=C2,5.647557,0.573704,5.647557,0.573704,0.771297,199.253,186.149,199.099714,76,...,0,0,0,0,0,0,0,0,0,0
552,CC(C)SSC(C)C,2.222269,0.766111,2.222269,0.766111,0.566681,150.312,136.200,150.053692,50,...,0,0,0,0,0,0,0,0,0,0


In [25]:
rdkit_descriptors.to_csv('rdkit_descriptors.csv')

In [28]:
# Finally, mordred descriptors
from mordred import Calculator, descriptors, error

In [29]:
# Initialise a calculator -- mordred works weirdly this way...
calc = Calculator(descriptors)

In [30]:
# Wow, many descriptors, much wow
len(calc.descriptors)

1826

In [31]:
mordred_descriptors = calc.pandas(rdkit_mols)

100%|█████████████████████████████████████████| 554/554 [00:14<00:00, 38.85it/s]


In [32]:
# It seems that unfortunately some descriptors cannot be computed. To filter this, 
# we find all columns that are of data type "object", since those contain non-numerical values usually.
error_columns = []
for i, e in enumerate(mordred_descriptors.dtypes):
    if e == 'object':
        error_columns += [i]
error_columns

[137,
 138,
 139,
 140,
 141,
 142,
 146,
 147,
 148,
 149,
 150,
 151,
 155,
 156,
 157,
 158,
 159,
 160,
 164,
 165,
 166,
 167,
 168,
 169,
 173,
 174,
 175,
 176,
 177,
 178,
 182,
 183,
 184,
 185,
 186,
 187,
 191,
 192,
 193,
 194,
 195,
 196,
 200,
 201,
 202,
 203,
 204,
 205,
 209,
 210,
 211,
 212,
 213,
 214,
 218,
 219,
 220,
 221,
 222,
 223,
 227,
 228,
 229,
 230,
 231,
 232,
 344,
 345,
 346,
 347,
 348,
 349,
 353,
 354,
 355,
 356,
 357,
 358,
 362,
 363,
 364,
 365,
 366,
 367,
 371,
 372,
 373,
 374,
 375,
 376,
 380,
 381,
 382,
 383,
 384,
 385,
 389,
 390,
 391,
 392,
 393,
 394,
 398,
 399,
 400,
 401,
 402,
 403,
 407,
 408,
 409,
 410,
 411,
 412,
 416,
 417,
 418,
 419,
 420,
 421,
 425,
 426,
 427,
 428,
 429,
 430,
 434,
 435,
 436,
 437,
 438,
 439,
 443,
 444,
 445,
 446,
 447,
 448,
 451,
 452,
 453,
 454,
 455,
 456,
 459,
 460,
 461,
 462,
 463,
 464,
 465,
 466,
 467,
 468,
 469,
 470,
 471,
 472,
 475,
 476,
 477,
 478,
 479,
 480,
 483,
 484,
 485

In [33]:
# use .drop to remove the affected columns 
mordred_descriptors = mordred_descriptors.drop(mordred_descriptors.columns[error_columns], axis = 1)

In [34]:
# and remove columns containing NA data, but I don't think this actually does anything...
mordred_descriptors = mordred_descriptors.dropna(axis = 1)

In [35]:
# again, insert first SMILES column
mordred_descriptors.insert(0, 'SMILE', smiles_list)
mordred_descriptors

,SMILE,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,N#CCC#N,2.828427,3.146264,0,0,5.464102,1.732051,3.464102,5.464102,1.092820,...,6.192362,25.583106,66.021798,9.431685,20,2,14.0,12.0,2.750000,1.500000
1,C1COCCO1,4.242641,4.000000,0,0,8.000000,2.000000,4.000000,8.000000,1.333333,...,7.627057,30.941317,88.052429,6.289459,27,3,24.0,24.0,1.500000,1.500000
2,CN(C)N,2.449490,2.449490,0,0,3.464102,1.732051,3.464102,3.464102,0.866025,...,6.188264,24.179697,60.068748,5.005729,9,0,12.0,9.0,3.111111,1.000000
3,C1=CC=CC(O)=C1C(=O)OC2=CC=CC=C2,12.158715,10.429609,0,0,20.519745,2.313044,4.626088,20.519745,1.282484,...,9.427466,47.906724,214.062994,8.233192,459,21,78.0,88.0,4.944444,3.638889
4,ClCCCCl,2.828427,3.146264,0,0,5.464102,1.732051,3.464102,5.464102,1.092820,...,6.192362,25.583106,111.984656,10.180423,20,2,14.0,12.0,2.750000,1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,C1=CC=CC=C1CCCCC,7.778175,7.057066,0,0,13.983360,2.156639,4.313279,13.983360,1.271215,...,8.491465,39.504541,148.125201,5.486119,182,10,46.0,48.0,3.361111,2.750000
550,CN(C)C1=CC=C(C=O)C=C1,7.956514,7.451864,0,0,13.619695,2.250875,4.501749,13.619695,1.238154,...,8.906935,40.567492,149.084064,6.776548,162,13,50.0,55.0,4.583333,2.611111
551,C1=C(N)C=CC=C1OCC2=CC=CC=C2,11.423098,9.399533,0,0,19.595004,2.241045,4.482091,19.595004,1.306334,...,9.206634,46.248795,199.099714,7.110704,412,17,72.0,79.0,4.083333,3.416667
552,CC(C)SSC(C)C,5.387307,5.580564,0,0,8.472136,2.000000,4.000000,8.472136,1.059017,...,7.738488,33.811160,150.053692,6.820622,74,5,30.0,28.0,4.722222,1.916667


In [36]:
mordred_descriptors.to_csv('mordred_descriptors.csv')

In [40]:
# finally, generate images of molecules
from rdkit.Chem import Draw
for i,mol in enumerate(rdkit_mols):
    Draw.MolToFile(mol, filename = 'molecular_images/' + str(i) + '.png')

In [4]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit import DataStructs
import numpy as np

In [9]:
maccs_mols = [MolFromSmiles(smiles) for smiles in smiles_list]

In [10]:
maccs_fingerprints = [MACCSkeys.GenMACCSKeys(mol) for mol in rdkit_mols]
maccs_fingerprints = np.asarray(maccs_fingerprints)

In [11]:
maccs_fingerprints.shape

(554, 167)

In [9]:
maccs_fingerprints = pd.DataFrame(data = maccs_fingerprints)
maccs_fingerprints.insert(0, 'SMILES', smiles_list)

In [10]:
maccs_fingerprints

,SMILES,0,1,2,3,4,5,6,7,8,...,157,158,159,160,161,162,163,164,165,166
0,N#CCC#N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,C1COCCO1,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,1,1,0
2,CN(C)N,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,0,0,0,0,0
3,C1=CC=CC(O)=C1C(=O)OC2=CC=CC=C2,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,1,1,1,0
4,ClCCCCl,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,C1=CC=CC=C1CCCCC,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,1,0
550,CN(C)C1=CC=C(C=O)C=C1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
551,C1=C(N)C=CC=C1OCC2=CC=CC=C2,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,1,1,1,1,0
552,CC(C)SSC(C)C,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [11]:
maccs_fingerprints.to_csv('maccs_fingerprints.csv')